<a href="https://colab.research.google.com/github/jfcasasp/ProyectoNPL/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bitsandbytes datasets accelerate loralib
!pip install git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
!pip install accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-gdjhxvqw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-gdjhxvqw
  Resolved https://github.com/huggingface/transformers.git to commit bd50402b56980ff17e957342ef69bd9b0dd45a7b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-79dilea5
  Running command git 

In [2]:
model_id = 'bertin-project/bertin-gpt-j-6B'

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/24.2G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [4]:
tokenizer.pad_token = tokenizer.eos_token

In [5]:
new_tokens = ["<SC>","<EC>"]

num_added_toks = tokenizer.add_tokens(new_tokens)
print("We have added", num_added_toks, "tokens")
# Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
model.resize_token_embeddings(len(tokenizer))

We have added 2 tokens


Embedding(50259, 4096)

In [6]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [8]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 7340032 || all params: 6057067603 || trainable%: 0.12118127914511903


In [9]:
import transformers

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pandas as pd
from datasets import load_dataset
from datasets import Dataset

In [16]:
data = pd.read_csv('/content/drive/MyDrive/sarcasmo.tsv', sep='\t')

In [27]:
dataset = Dataset.from_pandas(data)

In [28]:
dataset

Dataset({
    features: ['Locutor', 'Locución', 'Sarcasmo', 'Utsumi', 'hipérbole', 'Haverkate', 'Grice', 'Nakamura', 'Acto de habla'],
    num_rows: 959
})

In [29]:
def format_ds(example):
  example["Locución"] = "<SC>" + example['Locución'] + "<EC>"
  return example

In [30]:
dataset = dataset.map(format_ds, remove_columns=['Locutor', 'Utsumi', 'hipérbole', 'Haverkate', 'Grice', 'Nakamura', 'Acto de habla'])

Map:   0%|          | 0/959 [00:00<?, ? examples/s]

In [31]:
dataset['Sarcasmo'][0]

False

In [32]:
dataset = dataset.map(lambda samples: tokenizer(samples['Locución']), batched=True)

Map:   0%|          | 0/959 [00:00<?, ? examples/s]

In [35]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset['Sarcasmo'],
    eval_dataset=dataset['Locución'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=8,
        warmup_steps=100,
        max_steps=1000,
        learning_rate=2e-4,
        fp16=False,
        logging_steps=5,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [46]:
!pip install peft

In [53]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "mrm8488/bertin-gpt-j-6B-es-finetuned-chistes_spanish_jokes-1000"

# Cargar la configuración del modelo peft
config = PeftConfig(
    peft_type="lora",
    base_model_name_or_path="mrm8488/bertin-gpt-j-6B-es-finetuned-chistes_spanish_jokes-1000",
)

# Cargar el modelo de lenguaje generativo preentrenado
base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False, device_map='auto')

# Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# Combinar el modelo base y el modelo peft
model = PeftModel(base_model, config=config)

# Ejemplo de uso del modelo (puedes ajustar esto según tus necesidades específicas)
input_text = "Ejemplo de texto para generar continuación."
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"]
output = model.generate(input_ids)

# Imprimir la salida generada
print(tokenizer.decode(output[0], skip_special_tokens=True))



TypeError: ignored

In [54]:
def gen_joke(text):
  text = "<SC>" + text
  batch = tokenizer(text, return_tensors='pt')
  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=256, eos_token_id=50258)

  print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=False))

In [55]:
text = "Jaimito está"
gen_joke(text)

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50258 for open-end generation.




 <SC>Jaimito está en el colegio y le pregunta a la maestra: - ¿Qué es una célula? - Una célula es una célula. - ¿Y una célula es? - Una célula es una célula. - ¿Y una célula es? - Una célula es una célula. - ¿Y una célula es? - Una célula es una célula. - ¿Y una célula es? - Una célula es una célula. - ¿Y una célula es? - Una célula es una célula. - ¿Y una célula es? - Una célula es una célula. - ¿Y una célula es? - Una célula es una célula. - ¿Y una célula es? - Una célula es una c
